In [1]:
import pandas as pd
import os
import sys
import zipfile
import json
from tqdm.auto import tqdm
tqdm.pandas()
from ipywidgets import interact
import re
import pickle
util_dir = os.path.abspath('../utils')
sys.path.append(util_dir)
from utils import *

C:\Users\veldhuis\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Read the table with P numbers of tablets that belong to the Treasure Archive and Shoe Archive. Retrieve a list of numbers.

In [2]:
file = 'csv/treasury_shoes2.txt'
tr_df = pd.read_csv(file, encoding='utf8')
ids = list(tr_df['id_text'])
ids.sort()

If necessary, download the Ur3 JSON from epsd2. If the file is already in the directory `/jsonzip`, skip this step, but do assign 'epsd2/admin/ur3' to the variable `project`.

In [3]:
project = 'epsd2/admin/ur3'
#oracc_download([project]);

Function for parsing the JSON. Words that appear in year names are marked in the field `ftype`.

In [4]:
def parsejson(text):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            parsejson(JSONobject)
        if "label" in JSONobject:
            meta_d["label"] = JSONobject['label']
        if "f" in JSONobject:
            lemma = JSONobject["f"]
            if "ftype" in JSONobject:
                lemma['ftype'] = JSONobject['ftype'] # this picks up YN for year name
            lemma["id_word"] = JSONobject["ref"]
            lemma['label'] = meta_d["label"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
        if "strict" in JSONobject and JSONobject["strict"] == "1":
            lemma = {key: JSONobject[key] for key in dollar_keys}
            lemma["id_word"] = JSONobject["ref"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
    return

Call the parser for each of the P numbers in the list `ids`. The parser function needs the lists `lemm_l` and `dollar_keys`, as well as the dictionary `meta_d`. These objects are manipulated during the parsing process.

In [5]:
lemm_l = []
meta_d = {"label": None, "id_text": None}
dollar_keys = ["extent", "scope", "state"]
file = f"jsonzip/{project.replace('/', '-')}.zip"
try:
    z = zipfile.ZipFile(file) 
except:
    print(f"{file} does not exist or is not a proper ZIP file")
files = z.namelist() # list of all the files in the ZIP file
files = [name for name in files if name[-12:-5] in ids] # select only those of the treasury/leather archive
for filename in tqdm(files, desc = project):
    id_text = project + filename[-13:-5] 
    meta_d["id_text"] = id_text
    try:
        st = z.read(filename).decode('utf-8')
        data_json = json.loads(st)           
        parsejson(data_json)
    except:
        print(f'{id_text} is not available or not complete')
z.close()

epsd2/admin/ur3/P143238 is not available or not complete



The function `parsejson()` fills the lists `lemm_l` with data. Read this list into a DataFrame.

In [6]:
words = pd.DataFrame(lemm_l).fillna('')
keep = ['extent', 'scope', 'state', 'id_word', 'id_text', 'form', 'cf', 'gw', 'pos', 'ftype', 'label']
words = words[keep]

Remove comma's and spaces from Guide words

In [7]:
words['gw'] = words['gw'].replace([' ', ','], ['', ''], regex=True)

Simplify `id_text` ('P123456' instead of 'epsd2/admin/ur3/P123456') and add a field `id_line`.

In [8]:
words['id_text'] = [i[-7:] for i in words['id_text']]
words['id_line'] = [int(i.split('.')[1]) for i in words['id_word']]

In [9]:
proper_nouns = ['RN', 'PN', 'DN', 'AN', 'WN', 'ON', 'TN', 'CN', 'GN', 'SN']
physical_break = ['illegible', 'traces', 'missing', 'effaced']
logical_break = ['other', 'blank', 'ruling']
words['lemma'] = words["cf"] + '[' + words["gw"] + ']' + words["pos"]
words.loc[words["cf"] == "" , 'lemma'] = words['form'] + '[NA]NA'
words.loc[words["pos"] == "n" , 'lemma'] = words['form'] + '[]NU'
words.loc[words["state"].isin(logical_break), 'lemma'] = "break_logical"
words.loc[words["state"].isin(physical_break), 'lemma'] = "break_physical"
words.head(10)

,extent,scope,state,id_word,id_text,form,cf,gw,pos,ftype,label,id_line,lemma
0,n,line,missing,P104232.3,P104232,,,,,,,3,break_physical
1,,,,P104232.5.1,P104232,šu,šu,hand,N,,r 1,5,šu[hand]N
2,,,,P104232.5.2,P104232,ba-ti,teŋ,near,V/i,,r 1,5,teŋ[near]V/i
3,,,,P104232.6.1,P104232,ša₃,šag,heart,N,,r 2,6,šag[heart]N
4,,,,P104232.6.2,P104232,puzur₄-iš-{d}da-gan,Puzrišdagan,1,SN,,r 2,6,Puzrišdagan[1]SN
5,,,,P104232.7.1,P104232,iti,itud,moon,N,,r 3,7,itud[moon]N
6,,,,P104232.7.2,P104232,ki-siki-{d}nin-a-zu,Kisikininazuk,1,MN,,r 3,7,Kisikininazuk[1]MN
7,,,,P104232.7.3,P104232,min-kam,min,two,NU,,r 3,7,min[two]NU
8,,,,P104232.8.1,P104232,mu,mu,year,N,yn,r 4,8,mu[year]N
9,,,,P104232.8.2,P104232,si-mu-ru-um{ki},Si.mu.ru.um,00,SN,yn,r 4,8,Si.mu.ru.um[00]SN


Read list of name forms and Normalized names

In [10]:
normdf = pd.read_csv('Normalized/drehem_norm_names.csv', encoding='utf8')
normdf

,transliteration,normalization,remarks
0,A-AN-ba-az,A.AN.ba.az[]PN,MVN 13 464 r 10 (copy/photo)
1,A-KU-um,A.KU.um[]PN,"Aegyptus 10, 270 27 o 7 (copy)"
2,A-KU.KU-ta,A.KU.KU[]PN,AnOr 12 277 o i 17' (copy)
3,A-NI-ta,A.NI[]PN,Babyl. 7 pl. 22 18 o 3 (copy)
4,A-U.E₂-nu-tuku,A.U.E₂.nu.tuku[]PN,AnOr 07 150 o 2: A-U.KID-nu-tuku IŠ (copy/photo)
...,...,...,...
5802,{d}utu,Utu[]DN,NaN
5803,{d}utu-bar-ra,Utubara[]DN,NaN
5804,Ma₂-gur₈-mah,Magurmah[]ON,NaN
5805,Ma₂-dara₃-abzu,Madaraʾabzu[]ON,NaN


Download OGSL

In [11]:
oracc_download(['ogsl']);

Saving http://build-oracc.museum.upenn.edu/json/ogsl.zip as jsonzip/ogsl.zip.


List Ur 3 sign equivalencies

In [12]:
equiv = {'ANŠE' : 'GIR₃', 
        'DUR₂' : 'KU', 
        'NAM₂' : 'TUG₂', 
        'TIL' : 'BAD', 
        'NI₂' : 'IM',
        'ŠAR₂' : 'HI', 
        }
w = re.compile(r'\w+') # replace whole words only - do not replace TILLA with BADLA.
           # but do replace |SAL.ANŠE| with |SAL.GIR₃|

Parse OGSL

In [13]:
def parseogsljson(data_json):
    for key, value in data_json["signs"].items():
        key = re.sub(w, lambda m: equiv.get(m.group(), m.group()), key)
        if "values" in value:
            for n in value["values"]:
                d2[n] = key
    return

Create OGSL dictionary key = sign value, value = sign name.

In [14]:
d2 = {}  # this empty dictionary is filled by the parsejson() function, called in this cell.
file = "jsonzip/ogsl.zip"
z = zipfile.ZipFile(file) 
filename = "ogsl/ogsl-sl.json"
signlist = z.read(filename).decode('utf-8')
data_json = json.loads(signlist)                # make it into a json object (essentially a dictionary)
parseogsljson(data_json)  
with open('output/ogsl_dict.p', 'wb') as p:
    pickle.dump(d2, p)  

In [15]:
separators = ['{', '}', '-']
separators2 = ['.', '+', '|']  # used in compound signs
#operators = ['&', '%', '@', '×']
flags = "][?<>⸢⸣⌈⌉*/" # note that ! is omitted from flags, because it is dealt with separately
table = str.maketrans(dict.fromkeys(flags))

In [16]:
def signnames(translit):  
    """This function takes a string of transliterated cuneiform text and translates that string into a string of
    sign names, separated by spaces. In order to work it needs the variables separators, separators2, and table defined above. The variable table
    is used by the translate() method to translate all flags (except for !) to None. The function also needs a dictionary, called d2, that has as
    keys sign readings and sign names as corresponding values. In case a key is not found, the sign reading is replaced by itself."""
    signnames_l = []
    translit = translit.translate(table).lower()  # remove flags, half brackets, square brackets.
    translit = translit.replace('...', 'x')
    for s in separators: # split transliteration line into signs   
        translit = translit.replace(s, ' ').strip()
    s_l = translit.split() # s_l is a list that contains the sequence of transliterated signs without separators or flags
    s_l = [d2.get(sign, sign) for sign in s_l] # replace each transliterated sign with its sign name.
    # Now take care of some special situations: signs with qualifiers, compound signs.
    for sign in s_l:
        if '!' in sign: # corrected sign, as in ka!(SAG), get only the corrected reading.
            sign = sign.split('!(')[0]
            sign = sign.replace('!', '') # remove remaining exclamation marks
        elif sign[-1] == ')' and '(' in sign: # qualified sign, as in ziₓ(SIG₇) - get only the qualifier
            sign = sign.split('(')[1][:-1]
        if '×' in sign: #compound. Compound like |KA×NINDA| to be replaced by |KA×GAR|
            sign_l = sign.replace('|', '').split('×')
            #replace individual signs of the compound by OGSL names
            sign_l = [d2.get(sign, sign) for sign in sign_l] 
            # if user enters |KA*EŠ| this is transformed to ['KA', '|U.U.U|']. The pipes around U.U.U must be replaced by brackets
            sign_l = [f'({sign[1:-1]})' if len(sign) > 1 and sign[0] == '|' else sign for sign in sign_l]
            sign = f"|{'×'.join(sign_l)}|"  #put the sign together again with enclosing pipes.
        elif '.' in sign or '+' in sign: # using elif, so that compounds like |UD×(U.U.U)| are not further analyzed.
            for s in separators2:
                sign = sign.replace(s, ' ').strip() 
            sign_l = sign.split()  # compound sign split into multiple signs
            sign_l = [d2.get(sign, sign) for sign in sign_l]
            for se in separators2:   # in case d2.get returns a compound sign name
                sign_l = [si.replace(se, ' ').strip() for si in sign_l]
            signnames_l.extend(sign_l)
            continue
        sign = d2.get(sign, sign)
        signnames_l.append(sign)
    # add space before and after each line so that each sign representation is enclosed in spaces
    signnames = f" {' '.join(signnames_l).upper()} " 
    return signnames

Make a new field in the Normalized Names table, representing the sign sequence (sign names) of the transliteration of each name.

In [17]:
normdf["sign_names"] = normdf["transliteration"].progress_map(signnames)

Create a dictionary with `sign_names` as key and `normalization` as value.

In [18]:
normd2 = dict(zip(normdf['sign_names'], normdf['normalization']))

Add a column to `words` with the sequence of sign names for each word.

In [19]:
words['sign_names'] = words['form'].progress_map(signnames)

Use the `normd2` dictionary to transform sign name sequences into normalized names.

In [22]:
#words.loc[(words.pos.isin(proper_nouns + ['X'])) & (words.lemma.str.contains('.')), 
#          'lemma'] = words.progress_apply(lambda x: normd2.get(x['sign_names'], x['lemma']), axis=1)
words.loc[words.pos.isin(proper_nouns + ['X']), 'lemma'] = words.progress_apply(lambda x: normd2.get(x['sign_names'], x['lemma']), axis=1)

Use the (temporary) corrections.csv file to apply corrections to the lemmatization

In [23]:
corrections = pd.read_csv('Normalized/corrections.csv', encoding='utf8')
corr_d = dict(zip(corrections['form'], corrections['corr']))
words.loc[words.pos.isin(['PN', 'RN', 'X', 'DN']), 'lemma'] = words.progress_apply(lambda x: corr_d.get(x['form'], x['lemma']), axis =1)
words['pos'] = [w.split(']')[-1] if ']' in w else '' for w in words['lemma']]

In [24]:
texttype = []
for i in ids:
    ttype = ''
    text = list(words.loc[words.id_text == i, 'lemma'])
    if 'mu.DU[delivery]N' in text:
        ttype = 'intake'
    elif 'zig[rise]V/i' in text:
        ttype = 'expenditure'
    elif 'teŋ[near]V/i' in text: 
        ttype = 'transfer'
    texttype.append(ttype)
treasure = dict(zip(ids, texttype))

Select PNs (and RNs) but skip those that appear in Year Names or Seals. Add lugal (king) sukkalmah (prime minister) and nin (queen).

In [25]:
unnamed = {'lugal[king]N', 'nin[queen]N', 'sukkalmah[official]N'}
names = set(words.loc[words.pos.isin(['PN', 'RN', 'DN']), 'lemma'])
actors = unnamed | names
seal = list(words.loc[words.label.str.contains('seal'), 'id_word'])
yn = list(words.loc[words.ftype == 'yn', 'id_word'])
exclude = seal + yn
PNs = words.loc[(words.lemma.isin(actors)) & (~words.id_word.isin(exclude))].copy()

In [26]:
relation = {'dumu[child]N' : 'son of', 'adda[father]N' : 'father of',
            'ama[mother]N' : 'mother of', 'nin[sister]N' : 'sister of',
            'šeš[brother]N' : 'brother of', 'dam[spouse]N' : 'spouse of',
            'lu[person]N' : 'representative of', 'dumumunus[daughter]N' : 'daughter of',
           'lukur[priestess]N': 'concubine of'}
profession = ['nar[singer]', 'gala[singer]N', 'lugal[king]N', 'sukkal[secretary]N', 'šakkanak[general]N', 
             'sukkalmah[official]N', 'šagia[cup-bearer]N', 'azlag[fuller]N', 'nubanda[overseer]N']
places = [word for word in words['lemma'] if word[-2:] in ['SN', 'GN']] 
roles = {'dumu[child]N' : 'relative', 'adda[father]N' : 'relative',
        'ama[mother]N' : 'relative', 'nin[sister]N' : 'relative',
        'šeš[brother]N' : 'relative', 'dam[spouse]N' : 'relative',
        'lu[person]N' : 'relative', 'dumumunus[daughter]N' : 'relative',
        'lukur[priestess]N': 'relative', 'ŋiri[foot]N' : 'intermediary', 
        'arua[offering]N' : 'offerer', 'u[and]CNJ' : 'companion', 'lu[person]N' : 'superior', 
        'kišib[seal]N' : 'sealer', 'mu[name]N' : 'reason'}

First make sure that words before and after belong to the same text!

Check that indexes that are referenced belong to the indexes of that text with minimum and maximum.

In [36]:
role = []
attribute = []
no = []
for i in PNs.index:
    Pno = PNs.loc[i]['id_text']
    text = words.loc[words.id_text == Pno]
    mx = text.index[-1]
    mn = text.index[0]
    pre = words.loc[i-1]['lemma']
    r = ''
    a = ''
    n = 1
    for l in [relation, ['u[and]CNJ'], profession, places]:
        if i+n < mx:
            if text.loc[i+n]['lemma'] in l:
                a = f"{a} {text.loc[i+n]['lemma']}"
                n += 1
        else:
            break
    if words.loc[i+n]['lemma'] == 'maškim[administrator]N': 
        r = 'representative'
    elif words.loc[i+n]['lemma'] == 'zig[rise]V/i': 
        r = 'expender'
    elif words.loc[i+n]['lemma'] == 'mu.DU[delivery]N': 
        r = 'deliverer'
    elif i+n+1 <= mx:
        if f"{words.loc[i+n]['lemma']} {words.loc[i+n+1]['lemma']}" == 'šu[hand]N teŋ[near]V/i': 
            r = 'recipient'
        elif f"{words.loc[i+n]['lemma']} {words.loc[i+n+1]['lemma']}" == 'šu[hand]N us[follow]V/t': 
            r = 'sender'
    if i-1 >= mn and not r:
        r = roles.get(pre, r)
        if pre == 'ki[place]N' and words.loc[i+n-1]['form'].endswith('-ta'): 
            r = 'source'
        elif pre == 'ki[place]N' and words.loc[i+n-1]['form'].endswith('-še₃'): 
            r = 'destination'
        elif pre.endswith('PN') and PNs.loc[i]['lemma'] == 'sukkalmah[official]N':
            r = 'None'
        elif pre in profession:
            r = 'employer'
    if r == '' and treasure[Pno] == 'expenditure': 
        r = 'recipient'
    elif r == '' and treasure[Pno] == 'intake': 
        r = 'source'
    role.append(r)
    attribute.append(a)
    no.append(n)
PNs['role'] = role       
PNs['attribute'] = attribute 
PNs['no'] = no

In [37]:
anchor = '<a href="http://build-oracc.museum.upenn.edu/epsd2/admin/ur3/{}", target="_blank">{}</a>'
PNs2 = PNs.copy()
PNs2['id_word'] = [anchor.format(val,val) for val in PNs['id_word']]

In [38]:
@interact(rows = (1, len(PNs2), 1))
def showpns(rows = 25): 
    return PNs2[['id_word', 'form', 'pos', 'lemma', 'no', 'role', 'attribute']][:rows].style

interactive(children=(IntSlider(value=25, description='rows', max=1064, min=1), Output()), _dom_classes=('widg…

In [ ]:
intake = [i for i in ids if treasure[i] == 'intake']

In [ ]:

edges = []
for i in intake: 
    people = PNs.index[PNs.id_text == i]    
    d = dict(zip(PNs.loc[people]['role'], PNs.loc[people]['lemma']))
    for p in people:
        source = ''
        target = ''
        role = PNs.loc[p]['role']
        if role == 'intermediary':
            source = PNs.loc[p]['lemma']
            q = [n for n in people if n > p]
            if q:
                for r in q:
                    if PNs.loc[r]['role'] == 'recipient':
                        target = PNs.loc[r]['lemma']
                        break
        if role in ['sender', 'deliverer', 'source']:
            source = PNs.loc[p]['lemma']
            q = [n for n in people if n > p]
            if q:
                for r in q:
                    if PNs.loc[r]['role'] in ['recipient', 'intermediary']:
                        target = PNs.loc[r]['lemma']
                        break
        if source and target:
            edges.append([source, target, i])

In [ ]:
edgs = pd.DataFrame(edges)
edgs.columns = ['source', 'target', 'id_text']
anchor = '<a href="http://build-oracc.museum.upenn.edu/epsd2/admin/ur3/{}", target="_blank">{}</a>'
edgs2 = edgs.copy()
edgs2['id_text'] = [anchor.format(val,val) for val in edgs['id_text']]

In [ ]:
@interact(rows = (1, len(edgs2), 1))
def showedges(rows = 25): 
    return edgs2[:rows].style

In [ ]:
mudus = []
deliverer = ''
for i in intake:
    people = PNs.index[PNs.id_text == i].tolist()
    text = words.loc[words.id_text == i]
    mudu = text.index[text.lemma == 'mu.DU[delivery]N'].tolist()[0]
    p = [p for p in people if p < mudu]
    if len(p) > 0:
        deliverer = [PNs.loc[max(p), 'lemma'], i]
    else:
        deliverer = ['[...]', i]
    mudus.append(deliverer)     

In [ ]:
mudus

In [ ]:
intake[0]

In [ ]:
i = 'P103082'
text = words.loc[words.id_text == i]
people = PNs.index[PNs.id_text == i].tolist()
mudu = text.index[text.lemma == 'mu.DU[delivery]N'].tolist()[0]

In [ ]:
people

In [ ]:
PNs2.style

In [ ]:
role = []
attribute = []
no = []
for i in PNs.index:
    Pno = PNs.loc[i]['id_text']
    text = words.loc[words.id_text == Pno]
    mx = text.index[-1]
    mn = text.index[0]
    pre = words.loc[i-1]['lemma']
    r = ''
    a = ''
    n = 1
    if PNs.loc[i]['id_word'].split('.')[-1] == '1':
        attr = [w for w in text['id_word'] if w.split('.')[1] == PNs.loc[i]['id_word'][1]]
        n = len(attr)
    

In [43]:
PNs

,extent,scope,state,id_word,id_text,form,cf,gw,pos,ftype,label,id_line,lemma,sign_names,role,attribute,no
19,,,,P103277.4.1,P103277,{d}nin-gir₂-su,nin.gir₂.su,00,DN,,o 2,4,Ninŋirsuk[]DN,AN SAL TUG₂ GIR₂ SU,recipient,,1
20,,,,P103277.5.1,P103277,{d}ba-ba₆,Baba,1,DN,,o 3,5,Baʾu[]DN,AN BA U₂,recipient,,1
22,,,,P103277.6.2,P103277,lugal,lugal,king,N,,o 4,6,lugal[king]N,LUGAL,offerer,,1
24,,,,P103277.7.2,P103277,sipa-si-in,Sipa.si.in,00,PN,,o 5,7,ReʾiSuʾen[]PN,PA LU SI IN,intermediary,šagia[cup-bearer]N,2
27,,,,P103277.9.2,P103277,lu₂-dingir-ra-ta,Lu₂.dingir,00,PN,,r 1,9,Ludiŋirak[]PN,LU₂ AN RA TA,expender,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11439,,,,P512773.8.2,P512773,puzur₄-er₃-ra-ta,Puzur₄.er₃.ra,00,PN,,r 1,8,PuzurErra[]PN,|KA×GAN₂@T.ŠA| ARAD RA TA,expender,,1
11460,,,,P103022.4.2,P103022,lugal-mu,lugal,king,N,,o 2,4,lugal[king]N,LUGAL MU,recipient,,1
11462,,,,P103022.4.4,P103022,{d}šul-gi-ra-ka-še₃,šul.gi.ra,00,RN,,o 2,4,Šulgir[]RN,AN DUN GI RA KA EŠ₂,recipient,,1
11465,,,,P103022.6.1,P103022,u₂-ta₂-mi-šar-am,U₂.ta₂.mi.šar.ra.am,00,PN,,o 4,6,Utamišaram[]PN,U₂ DA MI SAR |GUD×KUR|,recipient,,1
